In [1]:
import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision, WriteOptions
from influxdb_client.client.write_api import SYNCHRONOUS
from flightsql import FlightSQLClient

from datetime import datetime

In [3]:
token = "whl_f4m7pZbnLdO6KHYmNFjFdJaGimywqZXMezcOCwFcwJyUOW0nomnbHXzMdrxf3TeKOGbzpUW4B2rDXgUu5Q=="
org = "csse4011"
url = "http://localhost:8086"

bucket="Demo_data"

write_client = influxdb_client.InfluxDBClient(url=url, token=token, org=org, debug=False)

write_api = write_client.write_api(write_options=SYNCHRONOUS)

In [4]:

point = (
    Point("Position")
    .field("X", 0)
    .field("Y", 0)
    .field("Z", 0)
    .field("Velocity", 2)
    )
write_api.write(bucket=bucket, org="csse4011", record=point)
                        

In [1]:
from scipy.ndimage import gaussian_filter
import numpy as np

x = np.array([1, 2, 3, 4, 5])
y = np.array([10, 20, 30, 40, 50])

# Combine the x and y coordinates into a single array
data = np.vstack((x, y))

# Apply Gaussian smoothing with a specified sigma (smoothing factor)
sigma = 1.0
smoothed_data = gaussian_filter(data, sigma=sigma)

# Separate the smoothed data back into x and y coordinates
smoothed_x = smoothed_data[0, :]
smoothed_y = smoothed_data[1, :]

# Print the smoothed coordinates
print(smoothed_x)
print(smoothed_y)

[ 5  8 12 15 18]
[ 8 13 20 26 31]


In [ ]:
import serial
import time
import threading
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import signal
import sys

import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision, WriteOptions
from influxdb_client.client.write_api import SYNCHRONOUS
from flightsql import FlightSQLClient
 
from datetime import datetime
from View2 import View
from sklearn.cluster import DBSCAN

import matplotlib.animation as animation
from matplotlib.artist import Artist

MAX_CSVS = 10
WRITE_GAP = 1
TEST_SAMPLES = 20

In [ ]:
EPSILON = 0.2

# Apply clustering algorithm
def clustering(self, testData):

    center_df = pd.DataFrame({'X': [0],'Y': [0],'Z': [0], 'label':[0]})
    separated_cluster = []
    total_cluster = pd.DataFrame({'X': 0, 'Y': 0, 'Z': 0,'cluster_num': 0}, index=[0])


    if(len(testData) > 0):

        data = testData.copy()
        x = data['X']
        y = data['Y']
        data = np.vstack((x, y))

        sigma = 1.0
        smoothed_data = gaussian_filter(data, sigma=sigma)

        # Create a db clustering algorithm
        db = DBSCAN(eps=EPSILON, min_samples=5).fit(smoothed_data)
        labels = db.labels_ 

        # What shape is the dictionary in that allows it to be accessed this way?
        n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise_ = list(labels).count(-1)

        # print("{0} {1}".format(n_clusters_, len(self.testData)))
        # print(self.testData)

        center_df = pd.DataFrame({'X': [0],'Y': [0],'Z': [0], 'label':[0]})

        for cluster_label in range(n_clusters_):

            # Iterate through, assign each point to a cluster and find the centre point of each
            # This is only going to return a two-dimensional array, which is no good
            cluster_points = smoothed_data[labels == cluster_label]
            cluster_points = cluster_points.assign(cluster_num=cluster_label)

            for _, row in cluster_points.iterrow():
                testRow = testData[testData["X"] == row['X'] and testData['Y'] == row['Y']]
                testRow['cluster_num'] = row['cluster_num']
                total_cluster = pd.concat([total_cluster, testRow], axis=0)

            # merged_df = pd.merge(cluster_points, testData, left_on=['X', 'Y'], right_on=['X', 'Y'], how='inner')
            # matching_rows = merged_df['X', 'Y']
            # cluster_points = testData[testData.isin(matching_rows.values).all(axis=1)]

            # total_cluster = pd.concat([total_cluster, cluster_points], axis=0)

            center_point = np.mean(cluster_points, axis=0)

            z_points = testData[testData['X'] > center_point['X'] - EPSILON and testData['X'] < center_point['X'] + EPSILON
                                and testData['Y'] > center_point['Y'] - EPSILON and testData['Y'] < center_point['Y'] + EPSILON]
            
            z = z_points['Z'].mean()

            entry = pd.Series({'X':center_point['X'], 'Y':center_point['Y'], 'Z':z, 'label':center_point['cluster_num']})
            center_df.loc[len(center_df)] = entry


            # These lines of code are an attempt to get the box grid working for up to five clusters
            #  check, but hopefully it works
            #----------------------------------------------------------------------------------------------

            x1, x2, y1, y2, z1, z2 = cluster_points['X'].min(), cluster_points['X'].max(), \
                    cluster_points['Y'].min(), cluster_points['Y'].max(), \
                    cluster_points['Z'].min(), cluster_points['Z'].max()
            
            self.draw3DRectangle_update(x1, x2, y1, y2, z1, z2, cluster_label)

        for i in range(n_clusters_, 5):
            self.draw3DRectangle_update(0, 0, 0, 0, 0, 0, i)

        #-------------------------------------------------------------------------------------------------
            
        
        self.num_clusters = n_clusters_
        self.cluster_points = center_df.drop(center_df.index[0])
        self.separated_clusters = total_cluster.drop(total_cluster.index[0])
